cechy wyodrębniane z warstwy granic miejscowosci - liczba ludnosci i powierzchnia

In [20]:
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler

przygotowanie warstwy prng - bazy tabeli na której będzie opierał się model

In [21]:
# miejscowości z prng - warstwa punktowa
miejscowosci_prng = gpd.read_file('PRNG_MIEJSCOWOSCI_SHP/PRNG_MIEJSCOWOSCI_SHP.shp')
miejscowosci_prng

,idIIP,idPRNG,nazwaGlown,elementRoz,elementRod,dopelniacz,przymiot,statusNazw,kategoria,rodzaj,...,jezykEgzon,latynEgzon,wojewodz,powiat,gmina,idGminy,poczWerOb,wersjaOb,waznaOd,geometry
0,PL.PZGiK.320.NGRP.2,2,Osiedle Struga,Struga,Osiedle,-la -gi,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,dolnośląskie,dzierżoniowski,Dzierżoniów-gmina miejska,0202021,2022-04-01T17:18:26,2022-04-01T17:18:26,2015-06-30,POINT (334530.330 319384.250)
1,PL.PZGiK.320.NGRP.4,4,Abisynia,Abisynia,NaN,-nii,NaN,niestandaryzowana,miejscowość,część wsi,...,NaN,NaN,wielkopolskie,pleszewski,Pleszew-gmina miejsko-wiejska,3020063,2022-04-01T17:20:18,2022-04-01T17:20:18,2016-04-25,POINT (419019.650 447421.450)
2,PL.PZGiK.320.NGRP.6,6,Abisynia,Abisynia,NaN,-nii,NaN,niestandaryzowana,miejscowość,przysiółek wsi,...,NaN,NaN,lubuskie,strzelecko-drezdenecki,Stare Kurowo-gmina wiejska,0806032,2023-04-25T17:23:30,2023-04-25T17:23:30,2016-01-27,POINT (274605.810 556427.550)
3,PL.PZGiK.320.NGRP.7,7,Abisynia,Abisynia,NaN,-nii,NaN,urzędowa,miejscowość,część wsi,...,NaN,NaN,wielkopolskie,czarnkowsko-trzcianecki,Drawsko-gmina wiejska,3002032,2022-03-22T21:26:58,2022-03-22T21:26:58,2016-03-10,POINT (300270.010 557521.480)
4,PL.PZGiK.320.NGRP.8,8,Abisynia,Abisynia,NaN,-nii,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,opolskie,krapkowicki,Krapkowice-gmina miejsko-wiejska,1605023,2022-04-01T17:18:26,2022-04-01T17:18:26,2019-10-09,POINT (427022.520 292751.830)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124558,PL.PZGiK.320.NGRP.269651,269651,Osiedle Ignacego Daszyńskiego,Ignacego Daszyńskiego,Osiedle,-la Ignacego Daszyńskiego,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,lubelskie,Lublin,Lublin-gmina miejska,0663011,2023-09-11T23:08:51,2023-09-11T23:08:51,2023-09-11,POINT (750806.120 383325.670)
124559,PL.PZGiK.320.NGRP.269652,269652,Osiedle Poręba,Poręba,Osiedle,-la Poręby,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,lubelskie,Lublin,Lublin-gmina miejska,0663011,2023-09-11T23:16:06,2023-09-11T23:16:06,2023-09-11,POINT (744782.200 378872.240)
124560,PL.PZGiK.320.NGRP.269653,269653,Osiedle Widok,Widok,Osiedle,-la -ku,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,lubelskie,Lublin,Lublin-gmina miejska,0663011,2023-09-11T23:22:20,2023-09-11T23:22:20,2023-09-11,POINT (745551.260 378780.490)
124561,PL.PZGiK.320.NGRP.269654,269654,Osiedle Górki,Górki,Osiedle,-la Górek,NaN,niestandaryzowana,miejscowość,część miasta,...,NaN,NaN,lubelskie,Lublin,Lublin-gmina miejska,0663011,2023-09-11T23:28:14,2023-09-11T23:28:14,2023-09-11,POINT (746310.060 378878.540)


In [22]:
# miejscowosci_prng['rodzaj'].sort_values().unique()

In [23]:
# zliczenie_rodzajow = miejscowosci_prng.groupby('rodzaj')['idMscNd'].agg([("Liczba niesamodzielnych", lambda x: x.count()), ("Liczba samodzielnych", lambda x: x.isna().sum())])
# zliczenie_rodzajow.reset_index().sort_values(by=['Liczba samodzielnych', 'Liczba niesamodzielnych'], ascending=False)

In [24]:
# miejscowosci_prng_2 = miejscowosci_prng[miejscowosci_prng['idMscNd'].isna()]
# miejscowosci_prng_2 = miejscowosci_prng_2[~miejscowosci_prng_2['rodzaj'].isin({'miasto', 'osiedle', 'część miasta'})]
# tabela_licznosci = miejscowosci_prng_2.groupby(['wojewodz', 'rodzaj']).size().unstack(fill_value=0)
# tabela_procentow = tabela_licznosci.div(tabela_licznosci.sum(axis=1), axis=0) * 100
# suma = tabela_licznosci.sum(axis=1)
# tabela_licznosci['suma'] = suma
# tabela_procentow = tabela_procentow.round(2)
# tabela_procentow

In [25]:
# ograniczenie do warmińsko-mazurskiego
warminskie_prng = miejscowosci_prng[miejscowosci_prng['wojewodz'] == 'warmińsko-mazurskie']
warminskie_prng['rodzaj'].sort_values().unique()

array(['część kolonii', 'część miasta', 'część osady', 'część wsi',
       'kolonia', 'kolonia osady', 'kolonia wsi', 'leśniczówka', 'miasto',
       'osada', 'osada leśna', 'osada leśna wsi', 'osada wsi', 'osiedle',
       'przysiółek', 'przysiółek osady', 'przysiółek wsi', 'wieś'],
      dtype=object)

In [26]:
# stworzenie df z najistotniejszymi kolumnami
warminskie_prng = warminskie_prng[['idZewnetrz', 'rodzaj', 'nazwaGlown', 'idMscNd', 'nazwaMscNd', 'geometry']]

In [27]:
# zmiana nazw dla czytelności
warminskie_prng.rename(columns={'idZewnetrz': 'SIMC_id', 'nazwaGlown': 'nazwa', 'idMscNd': 'id_msc_nd', 'nazwaMscNd': 'nazwa_msc_nd'}, inplace=True)
# zmiana typów z object na int64
warminskie_prng['SIMC_id'] = pd.to_numeric(warminskie_prng['SIMC_id'], errors='coerce').astype('Int64')
warminskie_prng['id_msc_nd'] = pd.to_numeric(warminskie_prng['id_msc_nd'], errors='coerce').astype('Int64')

In [28]:
# wyłączenie miast i pochodnych z analizy
warminskie_prng = warminskie_prng[~warminskie_prng['rodzaj'].isin({'miasto', 'osiedle', 'część miasta'})]
warminskie_prng.dropna(subset=['SIMC_id'], inplace=True)
# pozbycie się ewentualnych duplikatów
warminskie_prng.drop_duplicates(inplace=True)

In [29]:
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry
71,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620)
73,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540)
79,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880)
85,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890)
92,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410)
...,...,...,...,...,...,...
124182,1044583,część wsi,Trzech Króli,844100,Świniarc,POINT (552390.880 620035.670)
124183,1044577,część wsi,Tama za Wodą,844117,Trzcin,POINT (549613.840 611421.500)
124184,1044407,część wsi,Pod Mątowo,844146,Zwiniarz,POINT (555161.830 618831.980)
124241,1045803,część wsi,Ostrzywilk,847280,Radomno,POINT (537562.224 628117.656)


In [30]:
# informacyjne wyświetlenie rodzajów
# warminskie_prng['rodzaj'].drop_duplicates()

In [31]:
# dodanie kolumny czy_samodzielna
warminskie_prng['czy_samodzielna'] = [True if pd.isna(id_msc_nd) else False for id_msc_nd in warminskie_prng['id_msc_nd']]
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna
71,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True
73,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True
79,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True
85,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True
92,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False
...,...,...,...,...,...,...,...
124182,1044583,część wsi,Trzech Króli,844100,Świniarc,POINT (552390.880 620035.670),False
124183,1044577,część wsi,Tama za Wodą,844117,Trzcin,POINT (549613.840 611421.500),False
124184,1044407,część wsi,Pod Mątowo,844146,Zwiniarz,POINT (555161.830 618831.980),False
124241,1045803,część wsi,Ostrzywilk,847280,Radomno,POINT (537562.224 628117.656),False


ograniczenie zbioru danych miejscowości tylko do tych, które mają adresację
wyekstraktowanie danych z warstwy adresów - liczba adresów

In [42]:
# pobranie punktów adresowych
punkty_adresowe = gpd.read_file("28/PRG_PunktyAdresowe_28.shp")
punkty_adresowe.sort_values(by='SIMC_id').head()

,TERYT,PNA,SIMC_id,SIMC_nazwa,ULIC_id,ULIC_nazwa,Numer,geometry
20655,280302,13-200,0113419,Burkat,NaN,NaN,37,POINT (576331.144 601424.345)
20330,280302,13-200,0113419,Burkat,NaN,NaN,6A,POINT (576346.545 601260.438)
20324,280302,13-200,0113419,Burkat,NaN,NaN,85,POINT (576685.443 601103.681)
20321,280302,13-200,0113419,Burkat,NaN,NaN,25C,POINT (576892.119 601234.371)
20317,280302,13-200,0113419,Burkat,NaN,NaN,54,POINT (576161.963 601413.796)


In [33]:
punkty_adresowe['SIMC_id'] = pd.to_numeric(punkty_adresowe['SIMC_id'], errors='coerce').astype('Int64')

In [34]:
# usunięcie adresów bez przypisania do simc (id miejscowości)
punkty_adresowe = punkty_adresowe.dropna(subset=['SIMC_id'])

zliczenie adresów dla miejscowości

In [35]:
punkty_adresowe_simc = punkty_adresowe['SIMC_id'].value_counts().reset_index()
punkty_adresowe_simc.rename(columns={'count': 'liczba_adresow'}, inplace=True)
punkty_adresowe_simc

,SIMC_id,liczba_adresow
0,964465,12782
1,932703,11603
2,964650,3651
3,977670,3464
4,930609,3316
...,...,...
3825,482097,1
3826,481650,1
3827,481689,1
3828,757938,1


In [38]:
# stworzenie setów zawierających numery simc z warstwy adresowej i miejscowosci prng
set_punkty_adresowe = set(punkty_adresowe['SIMC_id'])
set_prng = set(warminskie_prng['SIMC_id'])

In [39]:
# wyznaczenie częsci wspólnej - zależy nam tylko na miejscowościach z adresacją
simc_z_adresacja = set_prng & set_punkty_adresowe
len(simc_z_adresacja)

3778

In [40]:
# ograniczenie warstwy miejscowośąci tylko do tych z adresacją
warminskie_prng = warminskie_prng[warminskie_prng['SIMC_id'].isin(simc_z_adresacja)]
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna
71,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True
73,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True
79,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True
85,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True
92,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False
...,...,...,...,...,...,...,...
121305,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True
121320,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True
121737,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True
123350,1067147,osada,Niewiastowo,<NA>,NaN,POINT (545598.880 719970.260),True


In [41]:
warminskie_prng.sort_values(by='SIMC_id').head()

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna
8452,113419,wieś,Burkat,<NA>,NaN,POINT (576420.870 601441.750),True
8474,113431,wieś,Bursz,<NA>,NaN,POINT (579272.350 591378.940),True
17964,113448,wieś,Filice,<NA>,NaN,POINT (576493.650 602925.440),True
19292,113454,wieś,Gąsiorowo,<NA>,NaN,POINT (578289.000 615214.190),True
20403,113460,wieś,Gnojenko,<NA>,NaN,POINT (572803.900 592242.680),True


In [78]:
# pomocnicze zapisanie do pliku
# warminskie_prng.to_file("28/warminskie_prng_z_adresacja")

C:\Users\agnie\AppData\Local\Temp\ipykernel_24900\2581334392.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  warminskie_prng.to_file("28/warminskie_prng_z_adresacja")


dodanie kolumny z liczbą adresów

In [18]:
warminskie_prng = warminskie_prng.merge(punkty_adresowe_simc, how='left', on='SIMC_id')
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26
...,...,...,...,...,...,...,...,...
3773,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2
3774,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6
3775,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5
3776,1067147,osada,Niewiastowo,<NA>,NaN,POINT (545598.880 719970.260),True,3


wyekstraktowanie danych z warstwy granic miejscowości - liczby ludności i powierzchni

In [19]:
# wczytanie granic miejscowości - warstwa poligonowa
granice_miejscowosci = gpd.read_file('28/warminskie_granice_miejscowosci.shp')
granice_miejscowosci

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,EM_LOKID,EM_PNAZW,EM_WERID,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,area,layer,path,geometry
0,2801,2C98CCB7-E677-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0177,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474382,PL.ZIPIN.870,NaN,109502,PL.PZGiK.204.PRNG,NaN,2773713.13,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,2801,2C98CCB7-E663-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0171,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474324,PL.ZIPIN.870,NaN,99197,PL.PZGiK.204.PRNG,NaN,6744120.39,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,2801,2C98CCB7-E68D-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0196,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474442,PL.ZIPIN.870,NaN,126609,PL.PZGiK.204.PRNG,NaN,7996737.92,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,2801,2C98CCB7-E690-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01108,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474465,PL.ZIPIN.870,NaN,140116,PL.PZGiK.204.PRNG,NaN,3882361.42,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,2801,2C98CCB7-E66F-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01228,ADMS07,NaN,inapplicable,przyblizony,NaN,...,0474420,PL.ZIPIN.870,NaN,127660,PL.PZGiK.204.PRNG,NaN,129325.10,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4486,2862,2BF83540-7727-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.624,ADMS02,NaN,inapplicable,dokladny,NaN,...,0964502,PL.ZIPIN.5261,NaN,00000000-0000-0000-0000-000000174289,PL.PZGiK.320.NGRP,NaN,252852.44,PL.PZGiK.341.2862__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((600044.410 660927.390, 600046.960 66..."
4487,2862,2BF83540-772E-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.6211,ADMS02,NaN,inapplicable,dokladny,NaN,...,0964531,PL.ZIPIN.5261,NaN,00000000-0000-0000-0000-000000100051,PL.PZGiK.320.NGRP,NaN,1829590.94,PL.PZGiK.341.2862__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((599076.590 655343.340, 599104.600 65..."
4488,2862,2BF83540-772D-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.625,ADMS02,NaN,inapplicable,dokladny,NaN,...,0964519,PL.ZIPIN.5261,NaN,00000000-0000-0000-0000-000000058520,PL.PZGiK.320.NGRP,NaN,2654590.12,PL.PZGiK.341.2862__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((596373.400 656082.910, 596356.220 65..."
4489,2862,2BF83540-7729-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.627,ADMS02,NaN,inapplicable,dokladny,NaN,...,0964548,PL.ZIPIN.5261,NaN,00000000-0000-0000-0000-000000109589,PL.PZGiK.320.NGRP,NaN,78172.33,PL.PZGiK.341.2862__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((597088.690 655546.650, 597090.000 65..."


In [20]:
# zmiana typu i nazwy kolumny klucza
granice_miejscowosci['IDTERYTMSC'] = pd.to_numeric(granice_miejscowosci['IDTERYTMSC'], errors='coerce').astype('Int64')
granice_miejscowosci.rename(columns={'IDTERYTMSC': 'SIMC_id'}, inplace=True)

In [21]:
# set z simc występującymi w warstwie granic miejscowości
set_granice_miejscowosci = set(granice_miejscowosci['SIMC_id'])
len(set_granice_miejscowosci)

4482

stworzenie nowej warstwy pomocniczej - granice miejscowości z adresacją

In [22]:
set_granice_prng = simc_z_adresacja & set_granice_miejscowosci
len(set_granice_prng) # niektóre wartości simc się powtarzają, ale nie ma duplikatów w tabeli z granicami

3764

In [23]:
granice_miejscowosci_pomoc = granice_miejscowosci[granice_miejscowosci['SIMC_id'].isin(set_granice_prng)]
granice_miejscowosci_pomoc.drop_duplicates(inplace=True)
granice_miejscowosci_pomoc

C:\Users\agnie\AppData\Local\Temp\ipykernel_17996\3038510844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  granice_miejscowosci_pomoc.drop_duplicates(inplace=True)


,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,EM_LOKID,EM_PNAZW,EM_WERID,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,area,layer,path,geometry
0,2801,2C98CCB7-E677-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0177,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474382,PL.ZIPIN.870,NaN,109502,PL.PZGiK.204.PRNG,NaN,2773713.13,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,2801,2C98CCB7-E663-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0171,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474324,PL.ZIPIN.870,NaN,99197,PL.PZGiK.204.PRNG,NaN,6744120.39,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,2801,2C98CCB7-E68D-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0196,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474442,PL.ZIPIN.870,NaN,126609,PL.PZGiK.204.PRNG,NaN,7996737.92,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,2801,2C98CCB7-E690-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01108,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474465,PL.ZIPIN.870,NaN,140116,PL.PZGiK.204.PRNG,NaN,3882361.42,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,2801,2C98CCB7-E66F-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01228,ADMS07,NaN,inapplicable,przyblizony,NaN,...,0474420,PL.ZIPIN.870,NaN,127660,PL.PZGiK.204.PRNG,NaN,129325.10,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,2819,311304D1-499A-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.104,ADMS03,NaN,inapplicable,dokladny,1.0,...,765116,PL.ZIPIN.1943,NaN,00000000-0000-0000-0000-000000040858,PL.PZGiK.204.PRNG,NaN,47631769.37,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682986.450 703211.410, 682991.700 70..."
4461,2819,311304D1-4996-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.103,ADMS03,NaN,inapplicable,dokladny,1.0,...,770979,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000040171,PL.PZGiK.204.PRNG,NaN,22742342.71,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((667907.680 715869.080, 667852.960 71..."
4462,2819,311304D1-49C6-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.116,ADMS07,NaN,inapplicable,dokladny,1.0,...,771341,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000133740,PL.PZGiK.204.PRNG,NaN,3079021.72,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((669232.280 706589.380, 669260.150 70..."
4464,2819,311304D1-49D7-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.47,ADMS07,NaN,inapplicable,dokladny,1.0,...,755632,PL.ZIPIN.902,NaN,00000000-0000-0000-0000-000000026309,PL.PZGiK.204.PRNG,NaN,2833038.18,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682807.210 712243.900, 682802.600 71..."


In [24]:
# granice_miejscowosci_pomoc.to_file("28/warminskie_granice_miejscowosci_z_adresacja.shp")

In [25]:
granice_miejscowosci_mieszkancy = granice_miejscowosci_pomoc[['SIMC_id', 'LMIESZKANC']]
granice_miejscowosci_powierzchnia = granice_miejscowosci_pomoc[['SIMC_id', 'area']]

In [26]:
granice_miejscowosci_mieszkancy.dropna(inplace=True)
granice_miejscowosci_mieszkancy['LMIESZKANC'] = granice_miejscowosci_mieszkancy['LMIESZKANC'].astype('Int64')
granice_miejscowosci_mieszkancy.drop_duplicates(inplace=True)
granice_miejscowosci_mieszkancy

C:\Users\agnie\AppData\Local\Temp\ipykernel_17996\1033420425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  granice_miejscowosci_mieszkancy.dropna(inplace=True)
C:\Users\agnie\AppData\Local\Temp\ipykernel_17996\1033420425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  granice_miejscowosci_mieszkancy['LMIESZKANC'] = granice_miejscowosci_mieszkancy['LMIESZKANC'].astype('Int64')
C:\Users\agnie\AppData\Local\Temp\ipykernel_17996\1033420425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

,SIMC_id,LMIESZKANC
0,474382,7
1,474324,62
2,474442,186
3,474465,105
4,474420,19
...,...,...
4460,765116,335
4461,770979,169
4462,771341,48
4464,755632,43


In [27]:
granice_miejscowosci_powierzchnia.drop_duplicates(inplace=True)
granice_miejscowosci_powierzchnia

C:\Users\agnie\AppData\Local\Temp\ipykernel_17996\3114135313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  granice_miejscowosci_powierzchnia.drop_duplicates(inplace=True)


,SIMC_id,area
0,474382,2773713.13
1,474324,6744120.39
2,474442,7996737.92
3,474465,3882361.42
4,474420,129325.10
...,...,...
4460,765116,47631769.37
4461,770979,22742342.71
4462,771341,3079021.72
4464,755632,2833038.18


dodanie kolumny liczba ludnosci

In [28]:
warminskie_prng = warminskie_prng.merge(granice_miejscowosci_mieszkancy, how='left', on='SIMC_id')
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow,LMIESZKANC
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21,65
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69,255
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33,83
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13,142
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26,88
...,...,...,...,...,...,...,...,...,...
3774,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2,5
3775,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6,7
3776,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5,12
3777,1067147,osada,Niewiastowo,<NA>,NaN,POINT (545598.880 719970.260),True,3,4


dodanie kolumny powierzchnia

In [29]:
warminskie_prng = warminskie_prng.merge(granice_miejscowosci_powierzchnia, how='left', on='SIMC_id')
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow,LMIESZKANC,area
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21,65,5184091.39
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69,255,4841070.89
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33,83,2073808.18
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13,142,12131404.50
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26,88,264581.60
...,...,...,...,...,...,...,...,...,...,...
3784,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2,5,4442946.03
3785,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6,7,24333.51
3786,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5,12,704732.13
3787,1067147,osada,Niewiastowo,<NA>,NaN,POINT (545598.880 719970.260),True,3,4,607350.95


In [30]:
# zmiana nazwy dla ujednolicenia
warminskie_prng.rename(columns={'area': 'powierzchnia', 'LMIESZKANC': 'liczba_mieszkancow'}, inplace=True)

In [31]:
warminskie_prng.drop_duplicates(subset='SIMC_id', inplace=True)
warminskie_prng

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21,65,5184091.39
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69,255,4841070.89
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33,83,2073808.18
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13,142,12131404.50
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26,88,264581.60
...,...,...,...,...,...,...,...,...,...,...
3784,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2,5,4442946.03
3785,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6,7,24333.51
3786,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5,12,704732.13
3787,1067147,osada,Niewiastowo,<NA>,NaN,POINT (545598.880 719970.260),True,3,4,607350.95


In [32]:
bez_na = warminskie_prng.dropna(subset=['liczba_adresow', 'liczba_mieszkancow', 'powierzchnia'])
bez_na

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21,65,5184091.39
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69,255,4841070.89
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33,83,2073808.18
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13,142,12131404.50
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26,88,264581.60
...,...,...,...,...,...,...,...,...,...,...
3783,1044100,osada,Rozdroże,<NA>,NaN,POINT (590288.790 610513.940),True,76,429,944005.19
3784,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2,5,4442946.03
3785,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6,7,24333.51
3786,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5,12,704732.13


In [33]:
bez_na['rodzaj'].value_counts()

rodzaj
wieś                2385
osada                937
część wsi            120
kolonia               82
osada leśna           81
przysiółek wsi        62
przysiółek            16
przysiółek osady      16
kolonia wsi            6
osada wsi              6
część kolonii          2
osada leśna wsi        1
Name: count, dtype: int64

usunięcie niepotrzebnych kolumn

In [36]:
warminskie_prng = warminskie_prng.drop(['id_msc_nd', 'nazwa_msc_nd'], axis=1)
warminskie_prng

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia
0,149222,wieś,Adamowo,POINT (517072.190 698942.620),True,21,65,5184091.39
1,157948,wieś,Adamowo,POINT (522923.230 649553.540),True,69,255,4841070.89
2,117890,wieś,Adamowo,POINT (561486.270 590967.880),True,33,83,2073808.18
3,471403,wieś,Adamowo,POINT (630835.770 669115.890),True,13,142,12131404.50
4,149647,część wsi,Adamowo-Osiedle,POINT (521525.150 699758.410),False,26,88,264581.60
...,...,...,...,...,...,...,...,...
3784,1047050,osada,Janowo,POINT (565834.280 675757.940),True,2,5,4442946.03
3785,764660,kolonia,Wąglik-Kolonia,POINT (682736.170 644304.860),True,6,7,24333.51
3786,770672,kolonia,Zajdy,POINT (724720.600 686869.290),True,5,12,704732.13
3787,1067147,osada,Niewiastowo,POINT (545598.880 719970.260),True,3,4,607350.95


dodanie kolumn wziązanych z analizą zabudowy

In [34]:
dane_zabudowa = pd.read_csv('28/analiza_zabudowy_warminskie.csv', sep=';')
dane_zabudowa

,SIMC_id,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,budynkiHoteli,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,371,83569.42,1.583784,146,0,3,1,2,1,...,0,1,14,0.349610,1066.709097,907.926013,85.114678,5.595754,4490.197390,4490.197390
1,113431,12,3383.35,1.250000,7,0,0,0,0,0,...,0,0,0,0.196046,247.277883,170.333300,68.883354,10.068545,520.223242,520.223242
2,113448,229,30639.25,1.606987,104,0,2,0,2,0,...,0,0,0,0.212606,497.217195,401.666020,80.782810,6.621034,2849.556375,2849.556375
3,113454,97,17698.85,1.278351,59,0,1,1,0,0,...,1,1,2,0.235016,486.756717,374.794532,76.998328,5.007587,2204.703636,2204.703636
4,113460,48,5193.56,1.145833,30,0,0,0,0,0,...,0,0,0,0.336787,441.816209,284.333460,64.355597,5.033175,1253.942402,1253.942402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1063126,22,1601.74,1.636364,7,0,0,0,0,0,...,0,0,0,0.200409,137.338223,94.242414,68.620673,12.877561,304.774513,304.774513
3759,1064491,52,16319.27,1.423077,29,0,0,0,0,0,...,0,2,0,0.917561,193.946638,128.815451,66.417986,5.625767,614.792574,614.792574
3760,1064500,3,311.89,1.333333,2,0,0,0,0,0,...,0,0,0,1.179343,78.110162,77.477626,99.190200,24.087480,175.737876,175.737876
3761,1064516,7,430.60,1.428571,5,0,0,0,0,0,...,0,0,0,0.028865,51.428196,40.195020,78.157554,11.037517,111.790464,111.790464


In [37]:
warminskie_prng = warminskie_prng.merge(dane_zabudowa, how='left', on='SIMC_id')
warminskie_prng

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,149222,wieś,Adamowo,POINT (517072.190 698942.620),True,21,65,5184091.39,60.0,14715.06,...,0.0,0.0,0.0,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024
1,157948,wieś,Adamowo,POINT (522923.230 649553.540),True,69,255,4841070.89,147.0,25168.40,...,0.0,0.0,0.0,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003
2,117890,wieś,Adamowo,POINT (561486.270 590967.880),True,33,83,2073808.18,93.0,8551.89,...,0.0,0.0,0.0,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759
3,471403,wieś,Adamowo,POINT (630835.770 669115.890),True,13,142,12131404.50,118.0,32306.85,...,0.0,0.0,0.0,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913
4,149647,część wsi,Adamowo-Osiedle,POINT (521525.150 699758.410),False,26,88,264581.60,29.0,7311.39,...,0.0,0.0,3.0,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,1047050,osada,Janowo,POINT (565834.280 675757.940),True,2,5,4442946.03,21.0,1600.72,...,0.0,0.0,0.0,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443
3774,764660,kolonia,Wąglik-Kolonia,POINT (682736.170 644304.860),True,6,7,24333.51,7.0,883.66,...,0.0,0.0,0.0,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274
3775,770672,kolonia,Zajdy,POINT (724720.600 686869.290),True,5,12,704732.13,20.0,2580.28,...,0.0,0.0,0.0,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269
3776,1067147,osada,Niewiastowo,POINT (545598.880 719970.260),True,3,4,607350.95,5.0,449.97,...,0.0,0.0,0.0,0.328823,81.384689,57.180786,70.259881,19.517409,180.381791,180.381791


badanie jak wygląda kwestia braków danych

In [38]:
bez_na2 = warminskie_prng.dropna()
bez_na

,SIMC_id,rodzaj,nazwa,id_msc_nd,nazwa_msc_nd,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia
0,149222,wieś,Adamowo,<NA>,NaN,POINT (517072.190 698942.620),True,21,65,5184091.39
1,157948,wieś,Adamowo,<NA>,NaN,POINT (522923.230 649553.540),True,69,255,4841070.89
2,117890,wieś,Adamowo,<NA>,NaN,POINT (561486.270 590967.880),True,33,83,2073808.18
3,471403,wieś,Adamowo,<NA>,NaN,POINT (630835.770 669115.890),True,13,142,12131404.50
4,149647,część wsi,Adamowo-Osiedle,149630,Władysławowo,POINT (521525.150 699758.410),False,26,88,264581.60
...,...,...,...,...,...,...,...,...,...,...
3783,1044100,osada,Rozdroże,<NA>,NaN,POINT (590288.790 610513.940),True,76,429,944005.19
3784,1047050,osada,Janowo,<NA>,NaN,POINT (565834.280 675757.940),True,2,5,4442946.03
3785,764660,kolonia,Wąglik-Kolonia,<NA>,NaN,POINT (682736.170 644304.860),True,6,7,24333.51
3786,770672,kolonia,Zajdy,<NA>,NaN,POINT (724720.600 686869.290),True,5,12,704732.13


In [39]:
bez_na2['rodzaj'].value_counts()
# czyli braki prezentują się prawie tak samo jak przed dodaniem cech zabudowy - dobry znak

rodzaj
wieś                2385
osada                936
część wsi            120
kolonia               82
osada leśna           81
przysiółek wsi        62
przysiółek            16
przysiółek osady      16
kolonia wsi            6
osada wsi              6
część kolonii          2
osada leśna wsi        1
Name: count, dtype: int64

In [43]:
na = warminskie_prng[warminskie_prng.isnull().any(axis=1)]
na
# rekordów z brakującymi danymi jest 65
# w związku z tym, że te braki na razie są ściśle związane z brakiem informacji o liczbie mieszkańców - można spróbować je uzupełnić później
# na razie braki w liczbie mieszkańców zostaną zastąpione zerami

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
41,488289,osada,Bajorki,POINT (665265.610 716079.110),True,1,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,473750,osada leśna,Barduń,POINT (575860.940 647622.450),True,2,<NA>,1274413.52,7.0,684.67,...,0.0,0.0,0.0,0.096342,142.343869,116.961328,82.168153,16.381119,314.344279,314.344279
144,486089,przysiółek wsi,Bieńki,POINT (647427.360 654896.520),False,2,<NA>,201594.06,18.0,1679.79,...,0.0,0.0,0.0,0.433279,171.640715,216.059073,125.878684,6.293846,617.169036,617.169036
320,490955,osada leśna,Nowe Chmielówko,POINT (542970.830 651975.800),True,1,<NA>,828537.59,18.0,1300.67,...,0.0,0.0,0.0,0.212041,58.080137,28.972878,49.884315,17.037495,122.385751,122.385751
339,484848,osada leśna,Cichogrąd,POINT (588119.010 643554.970),True,5,<NA>,3897072.99,6.0,707.13,...,0.0,0.0,0.0,0.438131,309.836649,416.116799,134.301994,20.592213,1022.086441,1022.086441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3724,477759,osada,Henrykowo,POINT (649628.660 682306.010),True,1,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3747,473885,osada,Barwiny,POINT (587705.710 649081.290),True,6,<NA>,1127197.07,10.0,2536.58,...,0.0,0.0,0.0,0.315121,449.123881,319.295185,71.092898,18.210738,1052.913971,1052.913971
3748,473773,osada,Cegłowo,POINT (584778.330 652306.700),True,5,<NA>,285089.87,9.0,1423.26,...,0.0,0.0,0.0,0.354005,112.064060,76.945263,68.661856,15.615158,259.986453,259.986453
3770,1061630,osada,Parkity,POINT (614332.880 690722.390),True,1,1,2194465.41,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
na_sum = warminskie_prng[warminskie_prng.isnull().sum(axis=1)>1]
print(na_sum)
# na razie zostaną wyrzucone rekordy, które mają więcej niż jeden brak danych - jest ich 15
set_na_sum = set(na_sum['SIMC_id'])
set_na_sum

      SIMC_id       rodzaj            nazwa                       geometry  \
41     488289        osada          Bajorki  POINT (665265.610 716079.110)   
694    148850        osada           Glinka  POINT (552501.570 717686.150)   
972    753604      kolonia          Jeglewo  POINT (696472.790 709470.410)   
980    766890  osada leśna            Jeleń  POINT (662381.110 639547.720)   
1659   155470         wieś           Łojewo  POINT (562394.900 710231.780)   
2139   755879        osada          Ostrowo  POINT (725014.650 718138.760)   
2768   756867        osada             Skup  POINT (721532.970 675348.720)   
2792   484759      kolonia         Smolanek  POINT (576410.850 643313.960)   
3333   148783         wieś            Wilki  POINT (562308.100 728620.860)   
3686  1067578         wieś    Mikosze-Osada  POINT (691453.040 665399.440)   
3699   479764         wieś  Kłusity Wielkie  POINT (588034.430 707217.590)   
3715   154186      kolonia           Gibity  POINT (546460.360 6

{148783,
 148850,
 154186,
 155470,
 477759,
 479764,
 484759,
 488289,
 753604,
 755879,
 756867,
 766890,
 1061630,
 1067526,
 1067578}

In [45]:
# wyrzucenie tych rekordów
warminskie_prng = warminskie_prng[~warminskie_prng['SIMC_id'].isin(set_na_sum)]
warminskie_prng

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,149222,wieś,Adamowo,POINT (517072.190 698942.620),True,21,65,5184091.39,60.0,14715.06,...,0.0,0.0,0.0,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024
1,157948,wieś,Adamowo,POINT (522923.230 649553.540),True,69,255,4841070.89,147.0,25168.40,...,0.0,0.0,0.0,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003
2,117890,wieś,Adamowo,POINT (561486.270 590967.880),True,33,83,2073808.18,93.0,8551.89,...,0.0,0.0,0.0,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759
3,471403,wieś,Adamowo,POINT (630835.770 669115.890),True,13,142,12131404.50,118.0,32306.85,...,0.0,0.0,0.0,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913
4,149647,część wsi,Adamowo-Osiedle,POINT (521525.150 699758.410),False,26,88,264581.60,29.0,7311.39,...,0.0,0.0,3.0,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1044100,osada,Rozdroże,POINT (590288.790 610513.940),True,76,429,944005.19,88.0,17862.33,...,0.0,1.0,0.0,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468
3773,1047050,osada,Janowo,POINT (565834.280 675757.940),True,2,5,4442946.03,21.0,1600.72,...,0.0,0.0,0.0,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443
3774,764660,kolonia,Wąglik-Kolonia,POINT (682736.170 644304.860),True,6,7,24333.51,7.0,883.66,...,0.0,0.0,0.0,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274
3775,770672,kolonia,Zajdy,POINT (724720.600 686869.290),True,5,12,704732.13,20.0,2580.28,...,0.0,0.0,0.0,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269


In [46]:
# sprawdzenie wartości null
# na = warminskie_prng[warminskie_prng.isnull().any(axis=1)]
# na

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
70,473750,osada leśna,Barduń,POINT (575860.940 647622.450),True,2,<NA>,1274413.52,7.0,684.67,...,0.0,0.0,0.0,0.096342,142.343869,116.961328,82.168153,16.381119,314.344279,314.344279
144,486089,przysiółek wsi,Bieńki,POINT (647427.360 654896.520),False,2,<NA>,201594.06,18.0,1679.79,...,0.0,0.0,0.0,0.433279,171.640715,216.059073,125.878684,6.293846,617.169036,617.169036
320,490955,osada leśna,Nowe Chmielówko,POINT (542970.830 651975.800),True,1,<NA>,828537.59,18.0,1300.67,...,0.0,0.0,0.0,0.212041,58.080137,28.972878,49.884315,17.037495,122.385751,122.385751
339,484848,osada leśna,Cichogrąd,POINT (588119.010 643554.970),True,5,<NA>,3897072.99,6.0,707.13,...,0.0,0.0,0.0,0.438131,309.836649,416.116799,134.301994,20.592213,1022.086441,1022.086441
389,470148,osada leśna,Czerwony Bór,POINT (614594.340 665570.150),True,2,<NA>,1215.94,2.0,226.10,...,0.0,0.0,0.0,1.078661,6.649149,6.649149,100.000000,13.298298,13.298298,13.298298
491,473709,kolonia wsi,Zaskwierki,POINT (578482.170 653824.260),False,4,<NA>,1374904.89,11.0,2530.64,...,0.0,0.0,0.0,0.123013,63.298820,40.333521,63.719230,5.868704,133.217309,133.217309
655,471700,osada leśna,Gęsikowo,POINT (628510.200 671160.770),True,3,<NA>,294460.39,4.0,594.89,...,0.0,0.0,0.0,0.173493,22.365852,14.555811,65.080511,20.582189,45.784765,45.784765
657,484268,osada leśna,Gibała,POINT (576058.990 635908.900),True,1,<NA>,749893.21,3.0,373.19,...,0.0,0.0,0.0,0.098263,19.930641,15.071780,75.621147,21.606985,37.603046,37.603046
772,476004,osada leśna,Grabowo Leśne,POINT (596091.960 599253.930),True,2,<NA>,1061489.83,3.0,319.10,...,0.0,0.0,0.0,0.210995,69.039783,64.709968,93.728522,32.140301,154.558465,154.558465
858,473649,osada,Guzowy Młyn,POINT (581669.690 648030.890),True,4,<NA>,675009.96,21.0,2029.96,...,0.0,0.0,0.0,0.355844,270.059580,193.093536,71.500347,6.990098,635.423290,635.423290


In [52]:
# tymczasowe uzupełnienie brakującej liczby mieszkańców w rekordach zerami
warminskie_prng_v_2 = warminskie_prng.copy()
warminskie_prng_v_2['liczba_mieszkancow'] = warminskie_prng['liczba_mieszkancow'].fillna(0)
warminskie_prng_v_2

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,149222,wieś,Adamowo,POINT (517072.190 698942.620),True,21,65,5184091.39,60.0,14715.06,...,0.0,0.0,0.0,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024
1,157948,wieś,Adamowo,POINT (522923.230 649553.540),True,69,255,4841070.89,147.0,25168.40,...,0.0,0.0,0.0,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003
2,117890,wieś,Adamowo,POINT (561486.270 590967.880),True,33,83,2073808.18,93.0,8551.89,...,0.0,0.0,0.0,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759
3,471403,wieś,Adamowo,POINT (630835.770 669115.890),True,13,142,12131404.50,118.0,32306.85,...,0.0,0.0,0.0,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913
4,149647,część wsi,Adamowo-Osiedle,POINT (521525.150 699758.410),False,26,88,264581.60,29.0,7311.39,...,0.0,0.0,3.0,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1044100,osada,Rozdroże,POINT (590288.790 610513.940),True,76,429,944005.19,88.0,17862.33,...,0.0,1.0,0.0,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468
3773,1047050,osada,Janowo,POINT (565834.280 675757.940),True,2,5,4442946.03,21.0,1600.72,...,0.0,0.0,0.0,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443
3774,764660,kolonia,Wąglik-Kolonia,POINT (682736.170 644304.860),True,6,7,24333.51,7.0,883.66,...,0.0,0.0,0.0,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274
3775,770672,kolonia,Zajdy,POINT (724720.600 686869.290),True,5,12,704732.13,20.0,2580.28,...,0.0,0.0,0.0,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269


In [53]:
# sprawdzenie wartości null
# na = warminskie_prng_v_2[warminskie_prng_v_2.isnull().any(axis=1)]
# na

,SIMC_id,rodzaj,nazwa,geometry,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep


dodanie kolumny z dlugością ulic

In [54]:
# pobranie ulic
ulice = gpd.read_file('28/PRG_Ulice_28.shp')
ulice['SIMC_id'] = pd.to_numeric(ulice['SIMC_id'], errors='coerce').astype('Int64')
ulice

,TERYT,SIMC_id,SIMC_nazwa,ULIC_id,Cecha,ULIC_nazwa,geometry
0,281707,490062,Spychowo,19830,Ulica,Sienkiewicza,"MULTILINESTRING ((655397.160 639786.350, 65548..."
1,281707,490062,Spychowo,33516,Ulica,Osiedle Zacisze,"LINESTRING (655270.026 639186.740, 655372.822 ..."
2,281707,490062,Spychowo,12555,Ulica,Mazurska,"MULTILINESTRING ((655548.711 638968.601, 65554..."
3,281707,490062,Spychowo,20254,Ulica,Słoneczna,"LINESTRING (655473.380 639464.090, 655631.960 ..."
4,281707,490062,Spychowo,10898,Ulica,Leśna,"LINESTRING (654907.620 639433.789, 654872.268 ..."
...,...,...,...,...,...,...,...
7575,281411,488906,Gryźliny,NaN,Ulica,Pasłęcka,"LINESTRING (591197.722 643305.848, 591148.426 ..."
7576,281411,488906,Gryźliny,12991,Ulica,Miodowa,"LINESTRING (588725.500 640508.320, 588770.600 ..."
7577,281411,488906,Gryźliny,17390,Ulica,Poziomkowa,"LINESTRING (589116.150 639631.470, 589114.200 ..."
7578,281411,488906,Gryźliny,26585,Ulica,Żurawinowa,"LINESTRING (590196.320 642044.460, 590209.020 ..."


In [55]:
ulice['dlugosc_ulic'] = ulice['geometry'].length

In [56]:
# łączna długość ulic
dlugosc_ulic = ulice[['SIMC_id', 'dlugosc_ulic']]
dlugosc_ulic = dlugosc_ulic.groupby('SIMC_id')['dlugosc_ulic'].sum().reset_index()
dlugosc_ulic # w metrach
# w przypadku ulic danych jest dużo mniej niż obserwacji, ze względu na to, że są to tylko "nazwane" ulice
# na razie nie będę ich dodawać, zobaczę, jak pójdzie z modelem w aktualnym stanie

,SIMC_id,dlugosc_ulic
0,116435,17858.676103
1,116458,8022.927889
2,116518,15928.923194
3,117914,1427.497986
4,118010,9400.601378
...,...,...
229,977947,49981.750589
230,983563,47577.358167
231,1063103,1871.085677
232,1066828,6818.686482


zapisanie do pliku tabeli, która będzie podstawą modelu

In [57]:
warminskie_prng_bez_geom = warminskie_prng_v_2.drop('geometry', axis=1)
warminskie_prng_bez_geom

,SIMC_id,rodzaj,nazwa,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,149222,wieś,Adamowo,True,21,65,5184091.39,60.0,14715.06,1.300000,...,0.0,0.0,0.0,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024
1,157948,wieś,Adamowo,True,69,255,4841070.89,147.0,25168.40,1.408163,...,0.0,0.0,0.0,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003
2,117890,wieś,Adamowo,True,33,83,2073808.18,93.0,8551.89,1.505376,...,0.0,0.0,0.0,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759
3,471403,wieś,Adamowo,True,13,142,12131404.50,118.0,32306.85,1.101695,...,0.0,0.0,0.0,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913
4,149647,część wsi,Adamowo-Osiedle,False,26,88,264581.60,29.0,7311.39,1.448276,...,0.0,0.0,3.0,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1044100,osada,Rozdroże,True,76,429,944005.19,88.0,17862.33,1.409091,...,0.0,1.0,0.0,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468
3773,1047050,osada,Janowo,True,2,5,4442946.03,21.0,1600.72,1.285714,...,0.0,0.0,0.0,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443
3774,764660,kolonia,Wąglik-Kolonia,True,6,7,24333.51,7.0,883.66,1.000000,...,0.0,0.0,0.0,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274
3775,770672,kolonia,Zajdy,True,5,12,704732.13,20.0,2580.28,1.000000,...,0.0,0.0,0.0,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269


In [59]:
# warminskie_prng_bez_geom.to_csv('28/warminskie_miejscowosci_do_analizy.csv', sep=';', header=True, index=False)